In [105]:
import os
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Set up transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the full dataset
full_dataset = datasets.ImageFolder(root="data/train", transform=transform)

# Create indices for training and testing split
dataset_size = len(full_dataset)
indices = list(range(dataset_size))
train_indices, test_indices = train_test_split(indices, test_size=0.25, stratify=[full_dataset.targets[i] for i in indices])

# Create subsets
train_dataset = Subset(full_dataset, train_indices)
test_dataset = Subset(full_dataset, test_indices)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Checking the classes
print("Classes:", full_dataset.classes)


Classes: ['Blackheads', 'Cystic', 'Papules', 'Pustules', 'Rosacea', 'Whiteheads']


In [106]:
import torch
import torch.nn as nn
from torchvision import models

# Set the number of classes based on your dataset
num_classes = len(train_loader.dataset.dataset.classes)

# Load the pre-trained ShuffleNetV2 model with x0.5 weights
model = models.shufflenet_v2_x0_5(pretrained=True)

# Modify the final fully connected layer to match the number of classes in your dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to the available device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


c:\Users\Nathan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Nathan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X0_5_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X0_5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [107]:
from sklearn.metrics import confusion_matrix
import numpy as np

def evaluate_per_class(model, test_loader):
    model.eval()  # Set to evaluation mode
    all_preds = []
    all_labels = []

    # Initialize a confusion matrix
    conf_matrix = np.zeros((len(test_loader.dataset.dataset.classes), len(test_loader.dataset.dataset.classes)))

    with torch.no_grad():  # No need to compute gradients
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Send to device
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            # Update confusion matrix
            for t, p in zip(labels.view(-1), preds.view(-1)):
                conf_matrix[t.item(), p.item()] += 1

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate accuracy for each class
    class_accuracies = []
    for i in range(len(test_loader.dataset.dataset.classes)):
        class_correct = conf_matrix[i, i]
        class_total = conf_matrix[i].sum()
        class_accuracy = class_correct / class_total if class_total > 0 else 0
        class_accuracies.append(class_accuracy)
        print(f"Accuracy for class {test_loader.dataset.dataset.classes[i]}: {class_accuracy * 100:.2f}%")
    
    # Calculate overall accuracy
    accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    return accuracy, class_accuracies


In [ ]:
# # Freeze all layers except the last one (the fully connected layer)
# for param in model.parameters():
#     param.requires_grad = False

# # Unfreeze the last fully connected layer
# for param in model.fc.parameters():
#     param.requires_grad = True


In [111]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [112]:
num_epochs = 10 # You can adjust this based on your dataset size and computational power

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")


Epoch 1/10, Loss: 1.6594
Epoch 2/10, Loss: 1.3436
Epoch 3/10, Loss: 1.1112
Epoch 4/10, Loss: 0.8977
Epoch 5/10, Loss: 0.7942
Epoch 6/10, Loss: 0.6317
Epoch 7/10, Loss: 0.4725
Epoch 8/10, Loss: 0.3699
Epoch 9/10, Loss: 0.2603
Epoch 10/10, Loss: 0.2423


In [102]:
# Get the class-wise accuracy and overall accuracy
overall_accuracy, class_accuracies = evaluate_per_class(model, test_loader)
print(f"Overall Test Accuracy: {overall_accuracy * 100:.2f}%")


Accuracy for class Blackheads: 65.00%
Accuracy for class Cystic: 66.67%
Accuracy for class Papules: 86.49%
Accuracy for class Pustules: 55.56%
Accuracy for class Rosacea: 90.00%
Accuracy for class Whiteheads: 57.89%
Overall Test Accuracy: 75.60%


# RL Testing

In [104]:
from PIL import Image
from torchvision import transforms

# Replace 'your_image.jpg' with the path to the image you want to classify
image_path = 'data/test/image.jpg'

# Define the image transformations (resize, normalization) for model input
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224 for DenseNet
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load and preprocess the image
image = Image.open(image_path)
image = transform(image).unsqueeze(0)  # Add a batch dimension

# Move the image tensor to the same device as the model
image = image.to(device)

# Set the model to evaluation mode
model.eval()

# Perform the forward pass
with torch.no_grad():
    outputs = model(image)

# Get the predicted class
_, predicted_class = torch.max(outputs, 1)

# Get the class label
class_label = train_loader.dataset.dataset.classes[predicted_class.item()]

print(f"The image is classified as: {class_label}")


The image is classified as: Whiteheads
